I have a csv file with all the date/times of ASTER observations I'm using, and I want to now download all the MODIS (MOD021KM) imagery for the same date/times covering the same spot.

I'm using the Earthdata API because doing this through their web portal would be a slow and cumbersome process.

Some references:
 * [Earthdata API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)
 * [MOD021KM product](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/MOD021KM/)

In [1]:
import requests
import pandas as pd
import io

I have a csv file with the specific timestamps for MODIS images that I want. (the timestamps are for coincident ASTER images)

In [2]:
# read in my csv file with the timestamps I want
timestamps = pd.read_csv('~/git/goes-cues/aster_AST_L1T_band14_Tuolumne-and-CUES.csv')
# preview the first few rows of this dataframe
timestamps.head()

,Unnamed: 0,timestampUTC,filepath,datetime
0,0,2017-03-04 18:51:03+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-03-04 10:51:03+00:00
1,1,2017-03-20 18:51:06+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-03-20 10:51:06+00:00
2,2,2017-04-21 18:51:07+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-04-21 10:51:07+00:00
3,3,2017-05-23 18:51:10+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-05-23 10:51:10+00:00
4,4,2017-06-07 06:07:10+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-06-06 22:07:10+00:00


Select just the first three rows for this example

In [3]:
timestamps = timestamps[:3]

In [4]:
# preview our selection
timestamps

,Unnamed: 0,timestampUTC,filepath,datetime
0,0,2017-03-04 18:51:03+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-03-04 10:51:03+00:00
1,1,2017-03-20 18:51:06+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-03-20 10:51:06+00:00
2,2,2017-04-21 18:51:07+00:00,/storage/spestana/ASTER/AST_L1T/geotiff/T/T_ba...,2017-04-21 10:51:07+00:00


Set up some of the granule search criteria for Earthdata search API

In [5]:
url = "https://cmr.earthdata.nasa.gov/search/granules.csv?" #"https://cmr.earthdata.nasa.gov/search/collections?has_granules_created_at\[\]=2015-01-01T19:00:00Z,"

# Product information for the MODIS products I want
shortname_id = "short_name=MOD021KM" # "short_name=MOD03"
version = "version=6.1"

# bounding box around the study area I want to look at (upper Tuolumne River basin)
# lower left longitude, lower left latitude, upper right longitude, upper right latitude
bounding_box = "bounding_box[]=-120,37,-118,38" 

In [6]:
# empty list to hold modis download URLs
modis_urls = []

for i, this_timestamp in enumerate(timestamps.timestampUTC):
       
    # the date is the first 10 characters of the timestamp string
    date = str(this_timestamp)[:10]
    # hour follows a space after the date
    hour_start = str(this_timestamp).split(" ")[1][:2]
    # and search until 1 hour later
    hour_end = int(hour_start) + 1
    
    start_time = "{date}T{hour_start}:00:00Z".format(date=date,hour_start=hour_start)
    end_time = "{date}T{hour_end}:00:00Z".format(date=date,hour_end=hour_end)
    time_range = "temporal=" + start_time + "," + end_time
    
    # build the whole request URL and make the request.get
    response = requests.get(url+"&"+shortname_id+"&"+version+"&"+bounding_box+"&"+time_range)
    
    # read the response CSV and put in a temporary dataframe
    df = pd.read_csv(io.StringIO(response.text))
    
    for modis_url in df['Online Access URLs']:
        modis_urls.append(modis_url)

We now have urls from EarthData for these files

In [7]:
modis_urls

['https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD021KM/2017/063/MOD021KM.A2017063.1850.061.2017312092219.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD021KM/2017/079/MOD021KM.A2017079.1850.061.2017313021934.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD021KM/2017/111/MOD021KM.A2017111.1850.061.2017314062948.hdf']

Write the list of URLs out to a text file:

In [8]:
with open("modis_download_list.txt", "w") as output:
    for row in modis_urls:
        output.write(str(row) + '\n')

**Download using wget and this text file list of all the MODIS observations**

`wget --http-user=YOUR_USERNAME --ask-password --keep-session-cookies --auth-no-challenge=on -c -i modis_download_list.txt`

---